In [164]:
from tkinter import *
import matplotlib.pyplot as plt
#bigger fonts
import matplotlib
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15)
matplotlib.rcParams.update({'font.size': 20})

matplotlib.use("TkAgg")

import pandas as pd

In [171]:
def Update_lbl():
    #Label(window, text= str(prop.get())).grid(row=0,column=4)
    lbl_prop.configure(text=str(prop.get()))
    
def clicked():
    lbl_prop.configure(text="READING PROPERTIES ...")
    T = pd.read_csv('../TOST_data/average_properties.txt',
            sep='\t',index_col=0,na_values=-99,
            dtype={'WEIGHT_GRAMS': float,
                   'GEST_AGE_DAYS':float,
                   'DELTA_SEC_MEASURE':float,
                   'MED_SPO2':float,
                   'MED_PI':float,
                   'MED_PR':float})
    lbl_prop.configure(text=T.columns.values)


window = Tk()
window.title("Prakash project NHS")
window.geometry('750x600')
lbl = Label(window,text='Select property to be plotted:',font=("Arial Bold", 15))#.grid(column=0,row=0)
lbl.grid(row=0,column=0)
prop = StringVar()
prop.set('MED_PR')
lbl_prop = Label(window, text= str(prop.get()))
lbl_prop.grid(row=7,column=0)
r1 = Radiobutton(window, text="Pulse Rate", variable=prop, value='MED_PR',indicatoron=0,command=Update_lbl).grid(row=0,column=1)
r2 = Radiobutton(window, text="Oxigen Saturation", variable=prop, value='MED_SPO2',indicatoron=0,command=Update_lbl).grid(row=0,column=2)
r3 = Radiobutton(window, text="Perfusion index", variable=prop, value='MED_PI',indicatoron=0,command=Update_lbl).grid(row=0,column=3)

btn = Button(window,text='READ DATA',bg='blue',fg='white',command=clicked)
btn.grid(column=0,row=6)



In [172]:
window.mainloop()